# Gist Memory Showcase (scottfalconer/gist-memory fork)

Welcome to the `gist-memory` showcase! This notebook provides a hands-on introduction to a specific fork (`scottfalconer/gist-memory`) of the `gist-memory` library. This version has had packaging fixes applied, simplifying its usage.

`gist-memory` helps you manage and compress text data (like dialogue history or large documents) to fit within the limited context windows of LLMs while retaining essential information.

This notebook will walk you through:
1.  **Setup**: Cloning the `scottfalconer/gist-memory` fork, changing into its directory, and installing the package along with dependencies.
2.  **Command Line Interface (CLI)**: A tour of the `gist-memory` CLI for quick operations.
3.  **Advanced Usage: Python Interface**: Using `gist-memory` programmatically for custom strategies and detailed experiments.
4.  **Conclusion**: Summary and next steps.

**Important Note on Notebook Execution**: This notebook uses `%cd` to change the current directory. Subsequent cells will operate from within the cloned `gist-memory` repository root.

Let's get started!

## 1. Initial Setup: Install Dependencies

First, we need to set up the environment. This involves cloning the `scottfalconer/gist-memory` fork, changing our current directory into it, installing the package from local source, and then downloading necessary dependencies and models.

**Note**: If you are running this notebook locally and have already cloned this specific fork and set up a virtual environment from within the repo root, you might be able to skip some of these steps. However, these cells are designed to work in a fresh Google Colab environment.

### 1.1 Clone `gist-memory` Repository (scottfalconer fork) and Change Directory

In [ ]:
# Clone the specified fork of the gist-memory repository
!git clone https://github.com/scottfalconer/gist-memory.git

# Change directory into the cloned repository
# Subsequent commands will run from the root of the 'gist-memory' repository
%cd gist-memory

Now that we are inside the `gist-memory` directory (from the `scottfalconer/gist-memory` fork), we can install the package and its dependencies using relative paths.

### 1.2 Install `gist-memory` Package from Local Source

In [ ]:
# Install the gist-memory package from the local source (current directory)
# The --no-build-isolation flag can be helpful if there are issues with 
# the build environment or specific package versions.
!pip install . --no-build-isolation

### 1.3 Install Dependencies from `requirements.txt`

Install the remaining dependencies listed in `requirements.txt` from the current directory. This ensures all features, including those for development and testing shown in this notebook, are available.

In [ ]:
!pip install -r requirements.txt

### 1.4 Download spaCy Model

Download the English language model from spaCy, which is used for text processing tasks like sentence segmentation. SpaCy should have been installed as part of the `requirements.txt`.

In [ ]:
!python -m spacy download en_core_web_sm

### 1.5 Set up PYTHONPATH (Usually Not Needed with `pip install .`)

When a package is installed using `pip install .` from its root directory, it's typically placed in the Python environment's `site-packages` directory and becomes accessible. Explicitly modifying `sys.path` is usually not necessary.

The following cell is commented out.

In [ ]:
# import sys
# import os
# sys.path.append(os.getcwd()) # os.getcwd() is now the repo root, if needed.

### 1.6 Download Pre-trained Models for `gist-memory`

Download the default embedding model (for text vectorization) and a small chat model (for response generation experiments) using the `gist-memory` CLI. The CLI should be available after the `pip install .` step.

In [ ]:
# Download the default sentence transformer model for embeddings
!gist-memory download-model --model-name all-MiniLM-L6-v2

In [ ]:
# Download a small chat model for demonstration purposes
!gist-memory download-chat-model --model-name tiny-gpt2

### 1.7 Configure Offline Usage (Optional)

If you have downloaded all necessary models and want to ensure the notebook runs without attempting to access Hugging Face Hub, you can set these environment variables. For this showcase, we'll leave them commented out.

In [ ]:
# For offline use after all models are downloaded, uncomment the following lines:
# import os
# os.environ['HF_HUB_OFFLINE'] = '1'
# os.environ['TRANSFORMERS_OFFLINE'] = '1'

## 2. Command Line Interface (CLI) Showcase

`gist-memory` also features a versatile Command Line Interface (CLI) for performing common operations without needing to write Python scripts. This is handy for quick tests, batch processing, model downloads, and direct interaction with compression strategies. The CLI became available after we installed `gist-memory` using `pip`.

Since we are operating from within the `gist-memory` repository root, file paths for the CLI should also be relative to this root.

### 2.1 Basic Help Command

To view all available CLI commands and their general options, use the `--help` flag.

In [ ]:
!gist-memory --help

### 2.2 Compression Command (`compress`)

The `compress` command lets you apply a compression strategy to various forms of input: a direct line of text, a single file, or an entire directory. The following subsections will demonstrate each of these use cases.

**Important Note**: The CLI uses strategies that are built into `gist-memory` or registered via its plugin system. The custom `TruncateStrategy` we defined and registered within this Python notebook session is *not* automatically available to the standalone CLI. For CLI examples, we'll use `truncate` if it's a built-in alias for a simple truncation strategy, or another standard built-in strategy like `gist` (which aims to create a gist of the text).

In [ ]:
# Example: Compress a short text string using the 'truncate' strategy (if available as a built-in/plugin)
# This command demonstrates compressing a simple line of text. 
# We use the 'truncate' strategy to keep the beginning of the text, fitting it within a 20-token budget.
!gist-memory compress --strategy truncate "This is a fairly long sentence that we want to compress using the command line interface to a small number of tokens." --budget 20

# Example: Compress a text file using the 'gist' strategy
# This command demonstrates compressing an entire file. We use 'sample_data/moon_landing/full.txt'.
# The 'gist' strategy is employed to summarize the content within a 100-token budget.
# The `compress` command directly handles file paths.
!gist-memory compress --strategy gist --file sample_data/moon_landing/full.txt --budget 100

# Example: Compress an entire directory using the 'gist' strategy
# This command demonstrates compressing all supported files within a directory.
# We use the 'sample_data/moon_landing' directory.
# The 'gist' strategy will be applied to each file, and the overall output might be a concatenation or structured representation.
# The `compress` command automatically detects that this is a directory and processes its contents.


In [ ]:
# Example: Compress an entire directory using the 'gist' strategy
# This command demonstrates compressing all supported files within a directory.
# We use the 'sample_data/moon_landing' directory.
# The 'gist' strategy will be applied to each file.
# The `compress` command directly handles directory paths.
!gist-memory compress --strategy gist sample_data/moon_landing --budget 200
# You might want to add --output some_directory_output.json if you want to inspect results later

#### Choosing a Strategy with `--strategy`

The `--strategy` option is key to controlling how `gist-memory` compresses your text. Different strategies have different behaviors:

*   **`gist`**: Aims to create a concise summary or gist of the text. (Already used in file/directory examples above).
*   **`truncate`**: Simply cuts the text to fit the token budget, usually keeping the beginning. (Already used in the text line example above).
*   **`passthrough`** (if available, or another simple built-in one): Might do minimal or no actual compression, useful for just token counting or as a baseline.

You can switch between them easily. For example, to compress our `full.txt` using `truncate` instead of `gist`:


In [ ]:
# Example of using a different strategy (truncate) on the full.txt file
# The `compress` command directly handles file paths.
!gist-memory compress --strategy truncate --file sample_data/moon_landing/full.txt --budget 100

Common options for `gist-memory compress`:
-   `--strategy <name>`: ID of the compression strategy (e.g., `gist`, `truncate`).
-   `--text "<string>"`: The text string to compress.
-   `--file <path>`: Path to a text file for compression (e.g., `sample_data/moon_landing/full.txt`).
-   `--budget <int>`: The target token budget.
-   `--tokenizer <name>`: (Optional) Specify a tokenizer if the strategy shouldn't use its default.
-   `--output <path>`: (Optional) Path to save the compressed output (e.g., as JSON). The compressed text is typically printed to the standard output if `--output` is not specified. Using `--output` is recommended for saving results, especially for larger inputs or when integrating into scripts.


### 2.3 Talk Command (`talk`)

The `talk` command enables interactive chat with an LLM that uses a specified compression strategy to manage the conversation history. This is great for qualitatively evaluating how different strategies affect conversation flow and context retention.

Running a fully interactive `talk` session is not feasible within a static notebook cell. Instead, we'll show how to get help for the command and a conceptual example of its use.

In [ ]:
!gist-memory talk --help

**Conceptual Usage of `talk`:**

You would typically run these in your terminal (from the repo root):

1.  **Talk with a strategy applied to the ongoing conversation history:**
    ```bash
    gist-memory talk --strategy gist --chat-model tiny-gpt2 --budget 150
    ```
    This starts an interactive session where `tiny-gpt2` is the LLM, and the `gist` strategy compresses the conversation history to about 150 tokens before each LLM call.

2.  **Talk using a pre-compressed memory file:**
    First, compress a document (using path relative to repo root):
    ```bash
    gist-memory compress --strategy gist --file ./sample_data/moon_landing/full.txt --budget 200 --output moon_memory.json
    ```
    Then, start a conversation using this memory as initial context:
    ```bash
    gist-memory talk --memory moon_memory.json --chat-model tiny-gpt2 --message "What were the main objectives?"
    ```

For a non-interactive check within the notebook, you can try sending a single message. The behavior might vary:


In [ ]:
# This attempts a non-interactive query using the 'truncate' strategy with a specified chat model.
# The CLI might provide a single response or indicate it's for interactive use.
!gist-memory talk --strategy truncate --chat-model tiny-gpt2 --message "What is the capital of France?" --budget 50

### 2.4 Other Useful CLI Commands

The `gist-memory` CLI offers other utilities. Here are a few, with examples of how to get their help messages:

-   `download-model`, `download-chat-model`: We used these in the setup to fetch models (e.g., `all-MiniLM-L6-v2`, `tiny-gpt2`).
-   `stats`: Provides statistics about datasets or memory files.
-   `validate`: Validates configuration files or memory formats against the library's schemas.

In [ ]:
!gist-memory stats --help
!echo "\n---------------------------------------\n" # Visual separator
!gist-memory validate --help

## 3. Advanced Usage: Python Interface

After exploring the CLI, this section delves into using `gist-memory` programmatically with Python for more advanced customization and evaluation.

### 3.1 Importing Modules for Python Usage

While the Command Line Interface (CLI) is excellent for quick tests and applying standard strategies, you'll want to use the Python interface for more advanced tasks such as defining custom compression logic, integrating `gist-memory` into your own Python applications, or conducting detailed programmatic evaluations.

To use `gist-memory` with Python for these purposes (as shown in the following sections), we first need to import the necessary modules. With the packaging fixes in the `scottfalconer/gist-memory` fork, imports should now use the standard package paths. `TokenBudget` has been removed due to import errors and usages changed to `int`.

In [ ]:
import os
import yaml # For pretty-printing experiment results
from pathlib import Path

# Core gist-memory components for experiments and strategy evaluation
from gist_memory import (
    ExperimentConfig,
    HistoryExperimentConfig,
    ResponseExperimentConfig,
    run_experiment,
    run_history_experiment,
    run_response_experiment,
    StrategyConfig # Now directly available or re-exported
)

# Components from submodules
from gist_memory.compression import (
    CompressionStrategy,
    CompressedMemory,
    CompressionTrace,
    TextBlock
    # TokenBudget removed due to ImportError
)

from gist_memory.constants import ONBOARDING_DEMO_DIR 
from gist_memory.registry import register_compression_strategy 
from gist_memory.utils import read_text, setup_logging 

# Initialize logging for cleaner output
setup_logging()

### 3.2 Defining and Using Custom Compression Strategies in Python

While the CLI is useful for applying pre-built strategies, the Python interface offers the power to define entirely new compression logic or precisely configure existing ones. At the heart of this is the `CompressionStrategy` class. This section demonstrates how to create and use your own custom strategy, giving you full control over the compression process.

#### 3.2.1 Define a Custom Compression Strategy

We'll create `TruncateStrategy`, a basic strategy that truncates text to a specified token budget. This demonstrates the fundamental structure of a `CompressionStrategy`:
- It inherits from `CompressionStrategy`.
- It has a unique `id`.
- It implements the `compress` method, which takes a `TextBlock` and an integer token budget, and returns a `CompressedMemory` object and a `CompressionTrace` object.

In [ ]:
class TruncateStrategy(CompressionStrategy):
    """A simple strategy that truncates text to the token budget."""
    id = "truncate" # Unique identifier for this strategy

    def __init__(self, config: StrategyConfig):
        super().__init__(config)

    def compress(self, text_block: TextBlock, budget: int) -> tuple[CompressedMemory, CompressionTrace]: # budget is now int
        """Compresses text by truncating to the budget."""
        input_summary = self.summarize(text_block) # Get summary of input
        
        # Simple truncation based on tokens
        tokens = self.tokenizer.encode(text_block.text)
        # budget is already an int, no need for budget.value
        truncated_tokens = tokens[:budget] # Slice tokens to meet budget 
        compressed_text = self.tokenizer.decode(truncated_tokens)
        
        # Create the CompressedMemory object
        compressed_memory = CompressedMemory(
            text=compressed_text,
            interaction_id=text_block.interaction_id, # Preserve interaction context
            source_block_ids=[text_block.id] # Track original source
        )
        output_summary = self.summarize(compressed_memory.to_text_block()) # Get summary of output
        
        # Create the CompressionTrace object for logging and analysis
        # Assuming self.config is a Pydantic model (StrategyConfig)
        strategy_config_dump = self.config.model_dump() if hasattr(self.config, 'model_dump') else self.config
        trace = CompressionTrace(
            strategy_name=self.id,
            strategy_config=strategy_config_dump, 
            token_budget=budget, # Pass the integer budget directly
            input_summary=input_summary,
            output_summary=output_summary,
            details={"truncation_details": "Tokens truncated from start"} # Strategy-specific details
        )
        return compressed_memory, trace

#### 3.2.2 Register the Custom Strategy

To make our `TruncateStrategy` available for use by its ID (e.g., in experiments), we register it with the library.

In [ ]:
register_compression_strategy(TruncateStrategy.id, TruncateStrategy)
print(f"Strategy '{TruncateStrategy.id}' registered successfully.")

#### 3.2.3 Load Sample Data

Let's load a sample text file. Since we've changed directory into `gist-memory`, paths are now relative to the repository root. We'll use an excerpt about the moon landing.

In [ ]:
# Define the path to the sample data file (relative to repo root).
sample_data_file_path = Path('sample_data/moon_landing/01_landing.txt')

# Load the content of the file
if sample_data_file_path.exists():
    sample_text = read_text(sample_data_file_path)
    print(f"Successfully loaded data from {sample_data_file_path}\n")
    print("First 300 characters of the sample data:\n")
    print(sample_text[:300])
else:
    print(f"Error: Sample data file not found at {sample_data_file_path}.")
    print(f"Current directory: {Path.cwd()}")
    # Fallback text if file not found, to allow notebook to continue
    sample_text = ("This is a fallback text because the sample data file was not found. "
                   "Please check the path. This text will be used for demonstration purposes instead.")

#### 3.2.4 Demonstrate Text Compression

Now, we'll use our `TruncateStrategy` to compress the loaded sample text. We need to:
1.  Create a `StrategyConfig` instance.
2.  Instantiate our `TruncateStrategy` with this config.
3.  Create a `TextBlock` from our sample text.
4.  Define an integer token budget.
5.  Call the `compress` method.

In [ ]:
# 1. Create a StrategyConfig for TruncateStrategy.
truncate_strategy_config_obj = StrategyConfig(name="truncate_demo_config", tokenizer_name="all-MiniLM-L6-v2")

# 2. Create an instance of the TruncateStrategy
truncate_compressor = TruncateStrategy(config=truncate_strategy_config_obj)

# 3. Define a TextBlock for compression
# TextBlock wraps the input text and associated metadata.
text_to_compress_block = TextBlock(id="moon_landing_excerpt", text=sample_text, interaction_id="demo_interaction_01")

# 4. Define an integer token budget (e.g., compress to a maximum of 75 tokens)
compression_budget_int = 75 

# 5. Compress the text
compressed_memory_output, compression_trace_output = truncate_compressor.compress(text_to_compress_block, compression_budget_int)

print(f"Compression complete using strategy '{truncate_compressor.id}'.")

#### 3.2.5 Understanding `CompressedMemory` and `CompressionTrace`

The `compress` method returned two important objects:

##### `CompressedMemory`

The `CompressedMemory` object encapsulates the result of the compression. Its most important attribute is `text`, which contains the compressed version of the input. It also stores metadata linking it to the original text and the interaction it belongs to.

In [ ]:
print("--- Compressed Text Output ---:\n")
print(compressed_memory_output.text)
print(f"\nOriginal Text Length (tokens): {compression_trace_output.input_summary.num_tokens}")
print(f"Compressed Text Length (tokens): {compression_trace_output.output_summary.num_tokens}")

##### `CompressionTrace`

The `CompressionTrace` object logs detailed metadata about the compression process. This is invaluable for debugging, analysis, and understanding the behavior of a strategy. It includes:
- The strategy name and its configuration.
- The token budget applied.
- Summaries of the input and output text (character counts, word counts, sentence counts, token counts).
- Any custom details logged by the strategy.

In [ ]:
print("--- Compression Trace Information ---:\n")
print(f"Strategy Name: {compression_trace_output.strategy_name}")
print(f"Strategy Configuration: {compression_trace_output.strategy_config}")
# Assuming CompressionTrace.token_budget can now store an int or was adapted
print(f"Token Budget: {compression_trace_output.token_budget} tokens") 
print(f"Input Summary: Chars={compression_trace_output.input_summary.num_chars}, Words={compression_trace_output.input_summary.num_words}, Sents={compression_trace_output.input_summary.num_sents}, Tokens={compression_trace_output.input_summary.num_tokens}")
print(f"Output Summary: Chars={compression_trace_output.output_summary.num_chars}, Words={compression_trace_output.output_summary.num_words}, Sents={compression_trace_output.output_summary.num_sents}, Tokens={compression_trace_output.output_summary.num_tokens}")
print(f"Strategy-Specific Details: {compression_trace_output.details}")

### 3.3 Evaluating Strategies and Testing Results in Python

Beyond applying strategies, it's crucial to evaluate their effectiveness. `gist-memory` provides a powerful experiment framework for systematically testing your custom strategies or different configurations of built-in ones. This allows you to gather metrics and compare performance for tasks like document ingestion, history retrieval, and response generation. This section explains how to set up and run these experiments.

#### Experiment Configuration Objects:
-   **`ExperimentConfig`**: Used for *ingestion experiments*. These evaluate how well a strategy compresses a single, potentially large, text document. Key metrics often include compression ratio, information preservation (if measurable via summarization or other techniques), and processing time.
-   **`HistoryExperimentConfig`**: Used for *history retrieval experiments*. These assess a strategy's ability to compress dialogue history while retaining information that is useful for retrieving relevant past turns or facts. Metrics typically involve retrieval scores like Mean Reciprocal Rank (MRR) or Recall@k against ground-truth relevant items.
-   **`ResponseExperimentConfig`**: Used for *response generation experiments*. These evaluate the quality of LLM-generated responses when the compressed dialogue history (produced by the strategy) is used as context. Evaluation can involve automated metrics (e.g., ROUGE, BLEU for summarization/translation tasks) or human evaluation.

#### 3.3.1 Run Ingestion Experiment

This experiment type evaluates how well a strategy compresses a single document. We'll use the `TruncateStrategy` and our moon landing text.

In [ ]:
# Path to the data for the ingestion experiment (reusing the sample data path, now relative to repo root)
ingestion_data_path = sample_data_file_path # Path('sample_data/moon_landing/01_landing.txt')

if ingestion_data_path.exists():
    # Create an ExperimentConfig for ingestion
    ingestion_exp_config = ExperimentConfig(
        id="ingestion_demo_experiment",
        experiment_type="ingestion",
        strategy_id=TruncateStrategy.id, 
        strategy_config=truncate_strategy_config_obj, 
        dataset_path=str(ingestion_data_path), 
        token_budgets=[50, 100], # Integer token budgets
        output_dir="./ingestion_experiment_output" 
    )

    # Run the ingestion experiment
    print(f"Running ingestion experiment: {ingestion_exp_config.id}")
    ingestion_metrics_results = run_experiment(ingestion_exp_config)

    # Print the metrics in a readable YAML format
    print("\n--- Ingestion Experiment Metrics ---:\n")
    print(yaml.safe_dump(ingestion_metrics_results, indent=2, sort_keys=False))
else:
    print(f"Error: Ingestion data file not found at {ingestion_data_path}. Current CWD: {Path.cwd()}")

The output above typically includes metrics like compression ratio, number of tokens before and after compression for each budget, and processing time.

#### 3.3.2 Run History Experiment

This experiment evaluates a strategy's ability to compress dialogue history for effective retrieval. It uses a dataset of dialogues where specific past turns are marked as relevant to current turns.

**Note**: The `truncate` strategy is very naive for history retrieval. More sophisticated strategies (like those using summarization or embedding similarity) would likely perform better here. This is for demonstration of the experiment mechanics.

In [ ]:
# Define the path to the history dialogues data (relative to repo root)
history_dialogues_data_path = Path('tests/data/history_dialogues.yaml')

if history_dialogues_data_path.exists():
    # Create a HistoryExperimentConfig
    history_exp_config = HistoryExperimentConfig(
        id="history_demo_experiment",
        experiment_type="history_retrieval",
        strategy_id=TruncateStrategy.id,
        strategy_config=truncate_strategy_config_obj, 
        dataset_path=str(history_dialogues_data_path),
        # param_grid allows testing different strategy parameters or budgets
        # Using integer for token_budget as TokenBudget class is not imported
        param_grid={"token_budget": [100, 150]},
        output_dir="./history_experiment_output"
    )

    # Run the history experiment
    print(f"Running history experiment: {history_exp_config.id}")
    history_experiment_results = run_history_experiment(history_exp_config)

    # Print the results
    print("\n--- History Experiment Results ---:\n")
    print(yaml.safe_dump(history_experiment_results, indent=2, sort_keys=False))
else:
    print(f"Error: History dialogues data not found at {history_dialogues_data_path}. Current CWD: {Path.cwd()}")

The results from a history experiment usually include retrieval metrics like MRR (Mean Reciprocal Rank) and Recall@k, indicating how well the compressed history helped in identifying relevant prior turns.

#### 3.3.3 Run Response Experiment

A response experiment assesses how compressed memory affects the quality of responses from an LLM. It uses a dataset of dialogues where the task is to generate a response based on the history.

**Note**: This experiment can take longer to run as it involves calls to an LLM (even a small local one like `tiny-gpt2`). The `truncate` strategy's impact on response quality might be detrimental if too much context is lost.

In [ ]:
# Define the path to the response dialogues data (relative to repo root)
response_dialogues_data_path = Path('tests/data/response_dialogues.yaml')

if response_dialogues_data_path.exists():
    # Create a ResponseExperimentConfig
    response_exp_config = ResponseExperimentConfig(
        id="response_demo_experiment",
        experiment_type="response_generation",
        strategy_id=TruncateStrategy.id, 
        strategy_config=truncate_strategy_config_obj, 
        dataset_path=str(response_dialogues_data_path),
        # Using integer for token_budget
        param_grid={"token_budget": [100]},
        chat_model_name="tiny-gpt2", # LLM for generating responses
        output_dir="./response_experiment_output"
    )

    # Run the response experiment
    print(f"Running response experiment: {response_exp_config.id}")
    # This can take a few minutes depending on the dataset size and model
    response_experiment_results = run_response_experiment(response_exp_config)

    # Print the results
    print("\n--- Response Experiment Results ---:\n")
    print(yaml.safe_dump(response_experiment_results, indent=2, sort_keys=False))
else:
    print(f"Error: Response dialogues data not found at {response_dialogues_data_path}. Current CWD: {Path.cwd()}")

Response experiment results often include metrics like ROUGE or BLEU (if reference responses are available and the task is suitable), or qualitative examples of generated responses. The output here will show file paths where detailed per-instance results and generated texts are stored.

## 4. Conclusion and Next Steps

This notebook has provided a tour of the `scottfalconer/gist-memory` fork, covering:
-   **Installation and Setup**: Cloning the specific fork, changing into its directory, and installing the package and dependencies locally.
-   **Command Line Interface (CLI)**: A tour of the `gist-memory` CLI for quick operations.
-   **Advanced Usage: Python Interface**: Using `gist-memory` programmatically for defining custom strategies, and the experiment framework.
-   **Conclusion**: Summary and next steps.

We hope this showcase helps you get started with this version of `gist-memory`!

### Further Resources:
-   **GitHub Repository (Fork Used)**: [https://github.com/scottfalconer/gist-memory](https://github.com/scottfalconer/gist-memory)
-   **Original GitHub Repository**: [https://github.com/google/gist-memory](https://github.com/google/gist-memory) (This notebook uses a fork, but the original may also be of interest).
-   **Documentation**: Check the `docs/` directory for more in-depth information.
-   **Examples**: Explore other examples in the `examples/` directory.

Feel free to open issues on the respective GitHub repositories for questions, bug reports, or feature requests.